# Data Cleaning
### Parsing, cleaning and structuring the journal data
### 1. Read in the data

In [ ]:
import pandas as pd
import numpy as np
import re
import pandas as pd
import re
from nltk.corpus import stopwords
import nltk

# Download stopwords if not already available
#nltk.download('stopwords')

# Get the list of English stop words and add custom ones
stop_words = set(stopwords.words('english'))
custom_stop_words = {'got', 'went', 'saw', 'made', 'played', 'home', 'drove', 'day', 'took'}
stop_words.update(custom_stop_words) 

# Path to your Excel file
file_path = "../data/raw/daily_sentences.xlsx"

# Read all sheets as a dictionary of DataFrames
sheets = pd.read_excel(file_path, sheet_name=None)
df_list = []
for sheet_name, df in sheets.items():
    df['Date'] = df['Date'].astype(str) + '/' + sheet_name # Adds the year to the date
    df_list.append(df)

# Combine all sheets into one DataFrame
df = pd.concat(df_list, ignore_index=True)
df = df[['Date', 'Sentence']] # Reduce columns

# Convert 'Date' column to datetime format
df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%Y', errors='raise')
df['Sentence'] = df['Sentence'].fillna('')
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.to_period('M')
df['Day'] = df['Date'].dt.day

### 2. Clean the data
Check for duplicates/NaNs - I accidently put '02-26' instead of '03-26' and put '11-27' instead of '11-28'

NOTE: This would need to be updated and improved if using on a different dataset

In [18]:
# Error checking for duplicate days and entries
duplicates = df[df.duplicated('Date', keep=False)]
#print(duplicates)

if len(duplicates) > 0:
    print(f"{len(duplicates)} duplicates found...")
    # There are two duplicates entires. One has the wrong day and one has the wrong month
    df.loc[84, 'Date'] = '2025-03-26'
    df.loc[922, 'Date'] = '2021-11-28'
    # Recheck for duplicates
    duplicates = df[df.duplicated('Date', keep=False)]
    print(f"{len(duplicates)} duplicates remaining")
else:
    print("No duplicates found")


# Checking for any null dates
print(df['Date'].isna().sum(), "null dates found")

4 duplicates found...
0 duplicates remaining
0 null dates found


Clean and standardize the text

In [24]:
pattern = re.compile(r'\b\w+\b')

# Function to clean, tokenize, and filter text
def tokenize_and_filter(text):
    if not isinstance(text, str):
        return ""
    words = pattern.findall(text.lower())
    return ' '.join([w for w in words if w not in stop_words])


df['cleaned_words'] = df['Sentence'].apply(tokenize_and_filter)
#df.head()

Another variation of cleaned text

In [22]:
import spacy
nlp = spacy.load("en_core_web_sm", disable=["ner", "parser"])

def clean_with_spacy(text):
    doc = nlp(text.lower())
    tokens = [token.lemma_ for token in doc 
              if not token.is_stop and token.is_alpha]
    return " ".join(tokens)

df['cleaned_words2'] = df['Sentence'].astype(str).apply(clean_with_spacy)

Output the results

In [25]:
df.to_excel("../data/cleaned/daily_sentences_cleaned.xlsx", index=False)